# Analysis of the White Fronted Goose (Anser albifrons) and Associated Subspecies

Author: Waley Wang


In [31]:
#
# Import nessesary libraries and do nessesary non-df related prepwork
#

import numpy as np
import pandas as pd

import warnings


# Supress warning related to data types (will get on first import of the csv file)
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

Import and trim relevant data. This notebook focuses only on data related to the Anser albifrons (species id: 1710) and its subspecies Anser albifrons elgasi (species id: 1719). This roughly corresponds to $222,322$ rows of data.

In [32]:
# Retrieve group 1 data from the relevant CSV file
goose_data = pd.read_csv('Bird_Banding_Data/NABBP_2023_grp_01.csv')

# Filter out all irrelevant species
goose_data = goose_data[(goose_data['SPECIES_ID'] == 1710) | (goose_data['SPECIES_ID'] == 1719)]

# Retrieve all relevant columns
goose_data = goose_data[['RECORD_ID', 
                         'EVENT_TYPE', 
                         'BAND', 
                         'ORIGINAL_BAND', 
                         'OTHER_BANDS', 
                         'EVENT_DATE', 
                         'EVENT_DAY', 
                         'EVENT_MONTH', 
                         'EVENT_YEAR', 
                         'SPECIES_ID', 
                         'ISO_COUNTRY', 
                         'ISO_SUBDIVISION', 
                         'LAT_DD', 
                         'LON_DD', 
                         'COORD_PREC', 
                         'BIRD_STATUS', 
                         'PERMIT', 
                         'BAND_STATUS', 
                         'BAND_TYPE']]

display(goose_data.count())

RECORD_ID          222322
EVENT_TYPE         222322
BAND               222322
ORIGINAL_BAND      222322
OTHER_BANDS           900
EVENT_DATE         222322
EVENT_DAY          222322
EVENT_MONTH        222322
EVENT_YEAR         222322
SPECIES_ID         222322
ISO_COUNTRY        222322
ISO_SUBDIVISION    220769
LAT_DD             222099
LON_DD             222099
COORD_PREC         222263
BIRD_STATUS        187292
PERMIT             187344
BAND_STATUS        185695
BAND_TYPE          222322
dtype: int64

A large number of the date cells (~ 5,000) do not work with the `pd.to_datetime()` function. Since this is vital information for the analysis, the below cell aims specifically to clean the dates and remove any unessesary columns after. The following is the process by which dates are chosen.

1. If the `'EVENT_DATE'` column already has a valid date that works with `pd.to_datetime()`, it will be the date used.
2. Otherwise if the `'EVENT_DAY'`, `'EVENT_MONTH'`, and `'EVENT_YEAR'` column all form a date that works with `pd.to_datetime()`, it will be the date used.
3. If neither of the above work, `NaT` will be assigned and the row will be dropped.

In [ ]:
#
# Clean time-related columns as described above.
#

# Attempt to apply pd.to_datetime() to the EVENT_DATE column.
goose_data['EVENT_DATE'] = pd.to_datetime(goose_data['EVENT_DATE'], format='%m/%d/%Y', errors='coerce')

# Assemble date guesses from the EVENT_MONTH, EVENT_DAY, and EVENT_YEAR columns.
dates_from_columns = pd.to_datetime(goose_data['EVENT_MONTH'].apply(str) + '/' + goose_data['EVENT_DAY'].apply(str) + '/' + goose_data['EVENT_YEAR'].apply(str), format='%m/%d/%Y', errors='coerce')

# Fill in all NaT values that can be filled with the guesses from the previous line.
goose_data['EVENT_DATE'] = goose_data['EVENT_DATE'].fillna(dates_from_columns)

# Remove all rows where EVENT_DATE is still NaT after the above operations.
goose_data = goose_data[goose_data['EVENT_DATE'].notna()]

# Drop the EVENT_MONTH, EVENT_DAY, and EVENT_YEAR columns as they are no longer needed.
goose_data = goose_data.drop(labels=['EVENT_MONTH', 'EVENT_DAY', 'EVENT_YEAR'], axis=1)

RECORD_ID                   int64
EVENT_TYPE                 object
BAND                       object
ORIGINAL_BAND              object
OTHER_BANDS                object
EVENT_DATE         datetime64[ns]
SPECIES_ID                  int64
ISO_COUNTRY                object
ISO_SUBDIVISION            object
LAT_DD                    float64
LON_DD                    float64
COORD_PREC                float64
BIRD_STATUS               float64
PERMIT                     object
BAND_STATUS               float64
BAND_TYPE                  object
dtype: object


Location data is also vital to analysis, so abit of cleaning will have to be done.

In [ ]:
#
# Clean the coordinates columns as described above.
#

# Filter out all rows where LAT_DD or LON_DD are NaN.
goose_data = goose_data[goose_data['LAT_DD'].notna() & goose_data['LON_DD'].notna()]

display(goose_data)

,RECORD_ID,EVENT_TYPE,BAND,ORIGINAL_BAND,OTHER_BANDS,EVENT_DATE,SPECIES_ID,ISO_COUNTRY,ISO_SUBDIVISION,LAT_DD,LON_DD,COORD_PREC,BIRD_STATUS,PERMIT,BAND_STATUS,BAND_TYPE
